In [1]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer, AutoModelForCausalLM
import json, os, shutil, re, random, io, requests, ctypes, sys, time, struct
from datasets import load_dataset,load_from_disk
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/vipuser/miniconda3/envs/openr1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 06-03 12:33:00 [__init__.py:239] Automatically detected platform cuda.


2025-06-03 12:33:02,448	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
import pandas as pd
file_path = "../GRPO_data/val_prompt2.csv"
# model_path = "/home/LLM/models/Qwen2.5-0.5B-Instruct"
df = pd.read_csv(file_path)

In [3]:
model_path = "/home/e_disk2/Schema_R1-1.5B/data/GRPO-schema-fix/checkpoint-68"
num_pre_Q = 1

In [4]:
sampling_params = SamplingParams(n=num_pre_Q, temperature= 0.4, max_tokens=2048)
vllm_gen = LLM(model=model_path, gpu_memory_utilization=0.9)
tokenizer = AutoTokenizer.from_pretrained(model_path)

INFO 06-03 12:33:14 [config.py:689] This model supports multiple tasks: {'score', 'embed', 'reward', 'generate', 'classify'}. Defaulting to 'generate'.
INFO 06-03 12:33:14 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-03 12:33:16 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='/home/e_disk2/Schema_R1-1.5B/data/GRPO-schema-fix/checkpoint-68', speculative_config=None, tokenizer='/home/e_disk2/Schema_R1-1.5B/data/GRPO-schema-fix/checkpoint-68', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observa

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:04<00:00,  4.56s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:04<00:00,  4.57s/it]



INFO 06-03 12:33:24 [loader.py:458] Loading weights took 4.71 seconds
INFO 06-03 12:33:25 [gpu_model_runner.py:1291] Model loading took 2.8876 GiB and 5.185799 seconds
INFO 06-03 12:33:38 [backends.py:416] Using cache directory: /root/.cache/vllm/torch_compile_cache/a88dd6f1e2/rank_0_0 for vLLM's torch.compile
INFO 06-03 12:33:38 [backends.py:426] Dynamo bytecode transform time: 12.90 s
INFO 06-03 12:33:40 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 06-03 12:33:48 [monitor.py:33] torch.compile takes 12.90 s in total
INFO 06-03 12:33:49 [kv_cache_utils.py:634] GPU KV cache size: 1,142,048 tokens
INFO 06-03 12:33:49 [kv_cache_utils.py:637] Maximum concurrency for 32,768 tokens per request: 34.85x
INFO 06-03 12:34:23 [gpu_model_runner.py:1626] Graph capturing finished in 33 secs, took 0.45 GiB
INFO 06-03 12:34:23 [core.py:163] init engine (profile, create kv cache, warmup model) took 57.56 seconds
INFO 06-03 12:34:23 [core_client.py:435] Core engi

In [5]:
system_prompt = """
You are a Schema linking assistant. Your task is to select appropriate table names and corresponding column names from the database in <database>...</database> based on the user's question in <question>...</question>. 
Follow this response template: <think>\nYour Thought Process (Why were these choices made?)\n</think>\n<answer>\n###Tables: (Related Table);\n###Columns: (Related Columns from Tables);\n</answer> 
"""
print(system_prompt)


You are a Schema linking assistant. Your task is to select appropriate table names and corresponding column names from the database in <database>...</database> based on the user's question in <question>...</question>. 
Follow this response template: <think>
Your Thought Process (Why were these choices made?)
</think>
<answer>
###Tables: (Related Table);
###Columns: (Related Columns from Tables);
</answer> 



In [6]:
def make_user_prompt(question, database_schema):
    return f"<question>{question}</question>\n<database>{database_schema}</database>"

In [7]:
def gen_answers(prompts):
    tip_text = []
    for x in prompts:
        tip_text.append(tokenizer.apply_chat_template([
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": x}], tokenize=False, add_generation_prompt=True))
    voutputs = vllm_gen.generate(tip_text, sampling_params, use_tqdm=False)
    answers = [];  ans_token_ids = []
    for v in voutputs:
        for z in v.outputs: 
            answers.append(z.text)
            ans_token_ids.append(z.token_ids)
    return answers, ans_token_ids

In [8]:
# from tqdm import tqdm
sample_list = []
correct_list = []
for index, row in tqdm(df.iterrows(), total=len(df)):
    correct_list.append(row["target_schema"])
    user_prompt = [make_user_prompt(row["question"],row["database_schema"])]
#     print(user_prompt)
#     print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    response = gen_answers(user_prompt)[0][0]
    sample_list.append(response)
    if index % 100 ==0 :
        print(response)
        print(row["target_schema"])
        print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

  0%|▏                                                                                                                                                                                                                                                  | 1/1004 [00:10<2:48:23, 10.07s/it]

<think>
Okay, let's tackle this question: "How many vocalists do we have?" The user wants to know the total number of vocalists available.

First, I need to identify which tables and columns in the database are relevant. The question is about counting vocalists, so the main focus should be on singers who perform vocals.

Looking at the database schema, there are four tables: stadium, singer, concert, and singer_in_concert. The question is about vocalists, so I should start by checking the singer table. The singer table has columns like Singer_ID, Name, Country, Song_Name, Song_release_year, Age, and Is_male. 

The key column here is probably the Singer_ID, which would link to the singer's presence in concerts. The concert table has concert_ID and concert_Name, but since the question is about counting singers, maybe the focus is on the singer's performance, not the concert details. 

The singer_in_concert table connects singers to concerts through concert_ID and Singer_ID. But the quest

 10%|████████████████████████▏                                                                                                                                                                                                                        | 101/1004 [15:54<2:27:40,  9.81s/it]

<think>
Okay, let's tackle this question step by step. The user wants to know the names of different car manufacturers who produced a vehicle in 1970. So, I need to figure out which tables and columns in the provided database schema are relevant here.

First, the question mentions "different car manufacturers," which might relate to the car_makers table. The car_makers table has columns like Id, Maker, FullName, and Country. To find manufacturers who produced vehicles in 1970, I need to link this table to the cars_data table, which has the Year column. 

Looking at the cars_data table, there's a Year column that can be used to filter for 1970. Then, the car_names table links to car_makers through MakeId, which is a foreign key. So, the key tables are car_makers, cars_data, and model_list. 

Wait, the model_list table has Maker and Model, but since we need to count distinct makers, maybe we don't need to join with model_list. The question specifically asks for manufacturers, not models,

 20%|████████████████████████████████████████████████▏                                                                                                                                                                                                | 201/1004 [31:52<2:02:58,  9.19s/it]

...

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                        | 901/1004 [2:21:22<16:26,  9.58s/it]

<think>
Okay, let me try to figure out how to answer the question "Which people owns the most dogs? List the people id, given name and family name." using the provided database schema.

First, the user wants the people who own the most dogs. So, I need to identify which tables and columns in the database are relevant here.

Looking at the tables, there's an 'Owners' table with columns like 'owner_id', 'first_name', 'last_name', 'street', 'city', 'state', 'zip_code', 'email_address', 'home_phone', and 'cell_number'. That seems like the obvious place to start. The question is about people owning dogs, so the 'Dogs' table might be involved. The 'Dogs' table has 'owner_id' which links to the 'Owners' table's 'owner_id'. 

The other tables like 'Breeds', 'Charges', 'Sizes', 'Treatment_Types', and 'Professionals' don't seem directly relevant to counting the number of dogs each owner has. They might have information about breeds, charges, sizes, treatments, or professionals, but not the count

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 1001/1004 [2:37:18<00:29,  9.90s/it]

<think>
Okay, let's see. The user's question is asking, "How many available characteristic are there in total?" So, I need to figure out which tables and columns in the provided database schema would hold that information.

First, the question mentions "available characteristic," which might refer to features or properties that are available for purchase or rent. Looking at the database, there's a table called `Other_Available_Features` which has columns like `feature_id`, `feature_type_code`, `feature_name`, and `feature_description`. That seems like a good starting point because it likely stores information about available features.

The other tables like `Ref_Feature_Types`, `Ref_Property_Types`, and `Properties` don't seem directly relevant to the count of available characteristics. They might have data about types of features or properties, but the question is about the number of features themselves, not their classifications or properties.

So, the key table here is `Other_Availa

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1004/1004 [2:37:49<00:00,  9.43s/it]


In [9]:
print(len(sample_list))
print(len(correct_list))
print(correct_list[0])

1004
1004
###Tables: singer;
###Columns: singer.name, singer.song_release_year, singer.country, singer.singer_id, singer.song_name, singer.age;



In [10]:
import pandas as pd
write_dict = {}
write_dict["sample"] = sample_list
write_dict["correct"] = correct_list

csv_filename = 'GRPO.csv'
# 将字典转换为 DataFrame
df = pd.DataFrame(write_dict)
 
# 写入 CSV 文件
df.to_csv(csv_filename, index=False)
 
print(f"字典已成功写入到 {csv_filename}")

字典已成功写入到 GRPO.csv


In [11]:
def check_socore_label(ground_truth ,answer):

    # get ground truth      
    ground_truth_tables_match = re.search(r'Tables:\s*(.*?);', ground_truth)
    ground_truth_columns_match = re.search(r'Columns:\s*(.*?);', ground_truth)
    
    truth_tables = set(ground_truth_tables_match.group(1).split(', ')) if ground_truth_tables_match else set()
    truth_columns = set([col.strip() for col in ground_truth_columns_match.group(1).split(', ')]) if ground_truth_columns_match else set()
    
    truth_tables = {s.lower() for s in truth_tables}
    truth_columns = {s.lower() for s in truth_columns}
    
    tables_match =  re.search(r'Tables:\s*(.*?);', answer)
    columns_match = re.search(r'Columns:\s*(.*?);', answer)
    pre_tables = set(tables_match.group(1).split(', ')) if tables_match else set()
    pre_columns = set([col.strip() for col in columns_match.group(1).split(', ')]) if columns_match else set()
    
    pre_tables = {s.lower() for s in pre_tables}
    pre_columns = {s.lower() for s in pre_columns}

    table_format_len = len(truth_tables - (truth_tables - pre_tables))
    column_format_len = len(truth_columns - (truth_columns - pre_columns))
    
    
    table_format_acc =  table_format_len / len(truth_tables)
    column_format_acc =  column_format_len / len(truth_columns)
    
   
    return table_format_acc, column_format_acc

In [12]:
def strict_format_label(ground_truth ,answer):

    # get ground truth      
    ground_truth_tables_match = re.search(r'Tables:\s*(.*?);', ground_truth)
    ground_truth_columns_match = re.search(r'Columns:\s*(.*?);', ground_truth)
    
    truth_tables = set(ground_truth_tables_match.group(1).split(', ')) if ground_truth_tables_match else set()
    truth_columns = set([col.strip() for col in ground_truth_columns_match.group(1).split(', ')]) if ground_truth_columns_match else set()
    
    
    tables_match =  re.search(r'Tables:\s*(.*?);', answer)
    columns_match = re.search(r'Columns:\s*(.*?);', answer)
    pre_tables = set(tables_match.group(1).split(', ')) if tables_match else set()
    pre_columns = set([col.strip() for col in columns_match.group(1).split(', ')]) if columns_match else set()
    
    table_format = 0
    column_format = 0
    if pre_tables ==  truth_tables:
        table_format = 1
    
    if pre_columns ==  truth_columns:
        column_format = 1
   
    return table_format, column_format

In [13]:
table_acc_list = []
columns_acc_list = []
for index in range(len(sample_list)):
#     print(correct_list[index])
#     print(sample_list[index])
#     print("++++++++++++++++++++++++++++++++")
#     break
    table_format_reward, column_format_reward = check_socore_label(correct_list[index],sample_list[index])
    print("table_format_reward:",table_format_reward)
    print("column_format_reward:",column_format_reward)
    table_acc_list.append(table_format_reward)
    columns_acc_list.append(column_format_reward)
    print("+++++++++++++++++++++++++++")
#     gen_len_reward(sample_list[index])

table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 0.6666666666666666
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_forma

In [14]:
print("avg_table_format:",np.sum(table_acc_list)/len(table_acc_list))
print("avg_column_format:",np.sum(columns_acc_list)/len(columns_acc_list))

avg_table_format: 0.9439741035856574
avg_column_format: 0.8184894351166762


In [15]:
table_format_list = []
column_format_list = []
for index in range(len(sample_list)):
#     print(correct_list[index])
#     print(sample_list[index])
#     print("++++++++++++++++++++++++++++++++")
#     break
    table_format, column_format = strict_format_label(correct_list[index],sample_list[index])
    print("table_format:",table_format)
    print("column_format:",column_format)
    table_format_list.append(table_format)
    column_format_list.append(column_format)
    print("+++++++++++++++++++++++++++")
#     gen_len_reward(sample_list[index])

table_format: 1
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 1
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 1
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 1
column_f

In [16]:
print("avg_table_format:",np.sum(table_format_list)/len(table_format_list))
print("avg_column_format:",np.sum(column_format_list)/len(column_format_list))

avg_table_format: 0.7320717131474104
avg_column_format: 0.3715139442231076


In [17]:
# dts-bench

In [18]:
def get_Table_columns(ground_truth ,answer):
    ground_truth_tables_match = re.search(r'Tables:\s*(.*?);', ground_truth)
    ground_truth_columns_match = re.search(r'Columns:\s*(.*?);', ground_truth)

    truth_tables = set(ground_truth_tables_match.group(1).split(', ')) if ground_truth_tables_match else set()
    truth_columns = set([col.strip() for col in ground_truth_columns_match.group(1).split(', ')]) if ground_truth_columns_match else set()
    truth_tables = {s.lower() for s in truth_tables}
    truth_columns = {s.lower() for s in truth_columns}

    tables_match =  re.search(r'Tables:\s*(.*?);', answer)
    columns_match = re.search(r'Columns:\s*(.*?);', answer)
    pre_tables = set(tables_match.group(1).split(', ')) if tables_match else set()
    pre_columns = set([col.strip() for col in columns_match.group(1).split(', ')]) if columns_match else set()
    pre_tables = {s.lower() for s in pre_tables}
    pre_columns = {s.lower() for s in pre_columns}
    return truth_tables, truth_columns, pre_tables, pre_columns

In [19]:
total_samples = len(sample_list)

table_total_accuracy = 0
table_filtered_accuracy = 0
table_total_precision = 0
table_total_recall = 0


columns_total_accuracy = 0
columns_filtered_accuracy = 0
columns_total_precision = 0
columns_total_recall = 0


for index in range(len(sample_list)):
#     print(correct_list[index])
#     print(sample_list[index])
#     print("++++++++++++++++++++++++++++++++")
#     break
    precision = 0
    recall = 0
    truth_tables, truth_columns, pre_tables, pre_columns = get_Table_columns(correct_list[index],sample_list[index])
    if set(pre_tables) == set(truth_tables):
        table_total_accuracy += 1
    table_true_positives = len(set(pre_tables) & set(truth_tables))
    table_false_positives = len(set(pre_tables) - set(truth_tables))
    table_false_negatives = len(set(truth_tables) - set(pre_tables))
    
    if table_true_positives == len(truth_tables):
        table_filtered_accuracy += 1
    
    if len(pre_tables) > 0:
        precision = table_true_positives / (table_true_positives + table_false_positives)
        recall = table_true_positives / (table_true_positives + table_false_negatives)
    
    table_total_precision += precision
    table_total_recall += recall
    
    ########################cloumns####################################
    precision = 0
    recall = 0
    if set(pre_columns) == set(truth_columns):
        columns_total_accuracy += 1
    columns_true_positives = len(set(pre_columns) & set(truth_columns))
    columns_false_positives = len(set(pre_columns) - set(truth_columns))
    columns_false_negatives = len(set(truth_columns) - set(pre_columns))
    
    if columns_true_positives == len(truth_columns):
        columns_filtered_accuracy += 1
    
    if len(pre_columns) > 0:
        precision = columns_true_positives / (columns_true_positives + columns_false_positives)
        recall = columns_true_positives / (columns_true_positives + columns_false_negatives)
    
    columns_total_precision += precision
    columns_total_recall += recall
    
    
    
    

In [20]:
# Calculate average precision and recall
avg_precision = table_total_accuracy / total_samples
avg_recall = table_total_recall / total_samples

# Calculate total accuracy
accuracy = table_total_accuracy / total_samples
filtered_accuracy = table_filtered_accuracy / total_samples

print("Total Accuracy:", accuracy)
print("Filtered Accuracy:", filtered_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)

Total Accuracy: 0.7320717131474104
Filtered Accuracy: 0.899402390438247
Average Precision: 0.7320717131474104
Average Recall: 0.9439741035856571


In [21]:
# Calculate average precision and recall
avg_precision = columns_total_accuracy / total_samples
avg_recall = columns_total_recall / total_samples

# Calculate total accuracy
accuracy = columns_total_accuracy / total_samples
filtered_accuracy = columns_filtered_accuracy / total_samples

print("Total Accuracy:", accuracy)
print("Filtered Accuracy:", filtered_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)

Total Accuracy: 0.38247011952191234
Filtered Accuracy: 0.6882470119521913
Average Precision: 0.38247011952191234
Average Recall: 0.8184894351166759
